In [4]:
import pandas as pd
dataset=pd.read_csv('Social_Network_Ads.csv')
dataset


,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset=pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset


,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset.drop('User ID', axis=1)


,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [7]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:

indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [10]:
dep=dataset[['Purchased']]
dep

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep,test_size=0.30,random_state=0)


In [12]:
from sklearn.preprocessing import StandardScaler
scx=StandardScaler()
X_train=scx.fit_transform(X_train)
X_test=scx.transform(X_test)

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid={
    'criterion':['gini','entropy','log_loss'],
    'n_estimators':[50,100],
    'max_features':['sqrt','log2',None]
}
grid=GridSearchCV(RandomForestClassifier(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None],
                         'n_estimators': [50, 100]},
             scoring='f1_weighted', verbose=3)

In [14]:
y_pred=grid.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 1], dtype=int64)

In [15]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score, roc_auc_score

cm=confusion_matrix(Y_test,y_pred)
print(cm)

clf_report=classification_report(Y_test,y_pred)
print(clf_report)

f1_score_weighted=f1_score(Y_test,y_pred, average='weighted')
print("f1 weighted value for best parameter {} {:.2f}".format(grid.best_params_, f1_score_weighted))


[[73  6]
 [ 4 37]]
              precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120

f1 weighted value for best parameter {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 100} 0.92


In [16]:
roc_score=roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])
print("roc_auc_score value for best parameter {} {:.2f}".format(grid.best_params_, roc_score))

roc_auc_score value for best parameter {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 100} 0.96


In [17]:
re = grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.190887,0.006648,0.026267,0.008040,gini,sqrt,50,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.874254,0.857143,0.823129,0.911105,0.909115,0.874949,0.033098,13
1,0.358553,0.003768,0.025270,0.006217,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.874254,0.838326,0.841398,0.911105,0.946153,0.882247,0.041421,6
2,0.183621,0.007229,0.017379,0.007241,gini,log2,50,"{'criterion': 'gini', 'max_features': 'log2', ...",0.874254,0.838326,0.841398,0.928571,0.926743,0.881858,0.039463,8
3,0.349558,0.004213,0.023569,0.005443,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.874254,0.840114,0.841398,0.947015,0.964286,0.893413,0.052554,1
4,0.184500,0.005824,0.024170,0.005492,gini,None,50,"{'criterion': 'gini', 'max_features': None, 'n...",0.855314,0.857143,0.824293,0.876368,0.927778,0.868179,0.034154,15
5,0.364485,0.008948,0.024402,0.001476,gini,None,100,"{'criterion': 'gini', 'max_features': None, 'n...",0.855314,0.821429,0.806153,0.858289,0.927778,0.853792,0.041984,18
6,0.192601,0.010974,0.017598,0.005050,entropy,sqrt,50,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.874254,0.857143,0.841398,0.911105,0.926743,0.882129,0.032172,7
7,0.363672,0.004342,0.022572,0.007210,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.892857,0.840114,0.859435,0.892857,0.964286,0.889910,0.042326,2
8,0.187036,0.006443,0.017436,0.007372,entropy,log2,50,"{'criterion': 'entropy', 'max_features': 'log2...",0.874254,0.875644,0.823129,0.911105,0.946153,0.886057,0.041101,4
9,0.364528,0.011936,0.029696,0.006454,entropy,log2,100,"{'criterion': 'entropy', 'max_features': 'log2...",0.874254,0.858503,0.823129,0.911105,0.909115,0.875221,0.032956,11


In [28]:
print(table.columns)

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_criterion', 'param_max_features', 'param_n_estimators', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score'],
      dtype='object')


In [30]:
print(table["mean_test_score"])

0     0.882303
1     0.878682
2     0.886010
3     0.886325
4     0.857059
5     0.860841
6     0.889381
7     0.885920
8     0.878804
9     0.893244
10    0.860427
11    0.868462
12    0.900426
13    0.878951
14    0.882498
15    0.893462
16    0.868055
17    0.864313
Name: mean_test_score, dtype: float64


In [31]:
if 'rank_test_score' in table.columns and 'mean_test_score' in table.columns :
    # Filter the row with the best rank (rank 1)
    best_row=table[table['rank_test_score']==1]

    # Print the mean_test_score for the best rank
    best_mean_test_score=best_row['mean_test_score'].values[0]

    print(f"Best mean test score for rank 1: {best_mean_test_score:.2f}")
else:
    print("Columns 'rank_test_score' or 'mean_test_score' not found in the DataFrame.")


Best mean test score for rank 1: 0.90
